# Interactive server with Jupyter and Klein

This creates a webserver step by step.

The first step is to install the
`asyncio`
kernel:

In [1]:
from twisted.internet import asyncioreactor
asyncioreactor.install(asyncio.get_event_loop())

It is important to do this first.
The Twisted native kernel gets installed
if you breath on Twisted too hard.

In [2]:
import asyncio
import json

from twisted.internet import reactor, endpoints
from twisted.web.server import Site
from klein import resource, route

After this was done,
import the modules that are actually being used.
Also useful to define a
`jsonify`
function
so that the route-specific code
can be simpler:

In [3]:
def jsonify(stuff):
    return json.dumps(stuff).encode("ascii")

Next,
define the
`endpoint`
on which klein will listen:

In [4]:
endpoint_description = "tcp:port=8080:interface=127.0.0.1"
endpoint = endpoints.serverFromString(reactor, endpoint_description)

Klein has more reasonable APIs.
but the simplest is the
"global" one.
This has the endpoint listening on a
`Site`
which serves
Klein's global resource.

In [5]:
endpoint.listen(Site(resource()))

<Deferred at 0xffff903848b0 current result: <<class 'twisted.internet.tcp.Port'> of <class 'twisted.web.server.Site'> on 8080>>

At this point,
the server is running.
It will return a
`404 Not Found`
for any route,
because it does not have any.
Try it!

Now,
a static value at the root:

In [6]:
@route("/")
def home(request):
    return jsonify(dict(value=5))

Going to
`http://localhost:8080`
will now return the json value.

Next,
a route that runs a dynamic computation:

In [8]:
@route("/len/<thing>")
def do_len(request, thing):
    global the_thing
    the_thing = thing
    return jsonify(dict(value=len(thing)))


Not sure why the length was returned like it was?
`the_thing`
global variable will hold the last value passed,
inspect as appropriate.

Dynamically change the root route
to return a value based on a global variable:

In [11]:
the_value = 78

In [12]:
@route("/")
def home(request):
    return jsonify(dict(value=the_value))

When changing the global variable,
the new value will be returned:

In [13]:
the_value = 13